### 深拷贝
##### 回顾js的数据类型： 
1. 七种基本： symbol number boolean string BigInt null undefine
 2. 引用类型Object 
        function Date Map Array Set等
- 对于原始类型 ,可以直接返回。
- 函数通常不需要深拷贝，此处保留引用
- 对于object，递归地拷贝里面的元素，注意
- 建议使用weakmap,键是弱引用，键所引用的对象被回收后会自动删除。 Map支持迭代，但强引用，可能导致内存泄漏
- Reflect.ownKeys可以遍历对象的所有属性，包括不可枚举属性和symbol
- Date/RegExp	必须特殊处理内部属性（如时间戳）和原型方法无法通过通用逻辑复制
- Map Set 是特殊的object, 也需要特殊处理

##### 回顾Map和普通object区别
- Map的Key可以是任意值，有序可迭代，没有默认原型； Object只能是字符串或者Symbol，不可迭代，有原型Object.prototype。

In [ ]:
function deepClone(target:any, map = new WeakMap()) {
  if (target === null || typeof target !== 'object' || 
    typeof target === 'bigint') return target;// null, undefined, number, string, boolean, symbol, bigint
  if (target instanceof Date) return new Date(target);
  if (target instanceof RegExp) return new RegExp(target);

  if (target instanceof Set) {
    const cloneSet = new Set();
    target.forEach(value => {
      cloneSet.add(deepClone(value, map)); // 深拷贝每个元素
    });
    return cloneSet;
  }
  
  if (target instanceof Map) {
    const cloneMap = new Map();
    target.forEach((value, key) => {
      cloneMap.set(deepClone(key, map), deepClone(value, map)); // 深拷贝键和值
    });
    return cloneMap;
  }

  if (map.has(target)) {
    return map.get(target);
  }

  const cloneTarget = Array.isArray(target) ? [] : Object.create(Object.getPrototypeOf(target));
  map.set(target, cloneTarget);

  Reflect.ownKeys(target).forEach(key => {
    if (typeof target[key] === 'function') {
      cloneTarget[key] = target[key]; // 保留函数引用
    } else {
      cloneTarget[key] = deepClone(target[key], map);
    }
  });

  return cloneTarget;
}


const originalMap = new Map();
originalMap.set('key1', { a: 1 });
originalMap.set('key2', new Set([1, 2, 3]));

const clonedMap = deepClone(originalMap);
console.log('Original Map:', originalMap);
console.log('Cloned Map:', clonedMap);


const originalSet = new Set();
originalSet.add({ b: 2 });
originalSet.add([4, 5, 6]);

const clonedSet = deepClone(originalSet);
console.log('Original Set:', originalSet);
console.log('Cloned Set:', clonedSet);

Original Map: Map(2) { "key1" => { a: 1 }, "key2" => Set(3) { 1, 2, 3 } }
Cloned Map: Map(2) { "key1" => { a: 1 }, "key2" => Set(3) { 1, 2, 3 } }
Original Set: Set(2) { { b: 2 }, [ 4, 5, 6 ] }
Cloned Set: Set(2) { { b: 2 }, [ 4, 5, 6 ] }
